In [2]:
# 时间数据的处理
import pandas as pd
time = pd.Timestamp('2025-5-21 10:11')
print(time)
print("类型",type(time))
print("年",time.year)
print("月",time.month)
print("天",time.day)
print("季度",time.quarter)
# 方法
print("是否是月底",time.is_month_end)
print("星期几：",time.day_name())
print("转换为天：",time.to_period("D"))
print("转换为季度：",time.to_period("Q"))
print("转换为月度：",time.to_period("M"))
print("转换为年度：",time.to_period("Y"))

2025-05-21 10:11:00
类型 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
年 2025
月 5
天 21
季度 2
是否是月底 False
星期几： Wednesday
转换为天： 2025-05-21
转换为季度： 2025Q2
转换为月度： 2025-05
转换为年度： 2025


In [3]:
import pandas as pd
import numpy as np

# 构造模拟数据（100条用户行为日志）
np.random.seed(42)
data = {
    "user_id": np.random.randint(1001, 1010, size=100),  # 9个用户
    "action_time": pd.date_range("2026-02-01 08:00:00", periods=100, freq="15min").astype(str), 
    "action_type": np.random.choice(["浏览", "下单", "支付", "退出"], size=100)
}
df = pd.DataFrame(data)

# 混入脏数据
df.loc[10,"action_time"] = "2026/02/01 10:30:00"  
df.loc[20,"action_time"] = "2026-02-01 12点30分" 

df.head()
# 处理异常时间：先清洗中文格式，再转datetime
# 步骤1：替换中文“点/分”为符号
df["action_time"] = df["action_time"].str.replace("点", ":").str.replace("分", "")
# 步骤2：转为datetime，无法转换的设为NaT（避免报错）
df["action_time"] = pd.to_datetime(df["action_time"], errors="coerce")
# 验证结果
print("转换后数据类型：", df["action_time"].dtype)
print("缺失的时间值数量：", df["action_time"].isnull().sum())  # 只有1个无法转换（2026-02-01 12:30 转换成功）

# 提取基础分量
df["year"] = df["action_time"].dt.year
df["month"] = df["action_time"].dt.month
df["day"] = df["action_time"].dt.day
df["hour"] = df["action_time"].dt.hour
weekday_map = {0:"周一",1:"周二",2:"周三",3:"周四",4:"周五",5:"周六",6:"周日"}
df["weekday"] = df["action_time"].dt.weekday.map(weekday_map)



转换后数据类型： datetime64[ns]
缺失的时间值数量： 2


In [7]:
# 时间筛选
df_filtered = df[df["action_time"].between("2026-02-01 10:00:00","2026-02-01 14:00:00")]
print(f"10:00-14:00的行为记录数：{len(df_filtered)}")
print(df_filtered[["user_id", "action_time", "action_type"]].head())

10:00-14:00的行为记录数：15
    user_id         action_time action_type
8      1005 2026-02-01 10:00:00          支付
9      1004 2026-02-01 10:15:00          支付
11     1008 2026-02-01 10:45:00          支付
12     1003 2026-02-01 11:00:00          支付
13     1006 2026-02-01 11:15:00          浏览
